<a href="https://colab.research.google.com/github/khanakshah27/models-/blob/main/why.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


!pip install torch torchvision opencv-python-headless

In [2]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{\n    "username": "khanakshah",\n    "key": "81605717adea65562910ff9f05c09b80"\n}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!pip install kaggle

In [5]:

import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
from torchvision.models.video import r3d_18, R3D_18_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:



import kagglehub
dataset_path = kagglehub.dataset_download("mission-ai/crimeucfdataset")
print("Dataset path:", dataset_path)


100%|██████████| 32.9G/32.9G [05:12<00:00, 113MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2


In [7]:

import os

dataset_path = dataset_path
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        print(os.path.join(root, file))

/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anomaly/Anomaly_Dataset/Anomaly_Videos/Temporal_Anomaly_Annotation_for_Testing_Videos.txt
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anomaly/Anomaly_Dataset/Anomaly_Videos/ReadMe-Anomaly-Detection.txt
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anomaly/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_926_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anomaly/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_871_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anomaly/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_722_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anomaly/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_224_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/anom

In [8]:
import os
import shutil


    # Construct the full paths for the files to keep
    # Reverting to hardcoded paths as requested
keep_files = [
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault036_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault018_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault027_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault026_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault037_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault014_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault039_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault006_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault051_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault003_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault042_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault020_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault007_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault012_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault017_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault023_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault031_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault050_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault008_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault040_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault034_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault046_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault028_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault041_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault025_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault032_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault048_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault024_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault049_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault045_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault022_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault010_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault013_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault044_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault005_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault011_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault035_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault004_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault019_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault016_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault033_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault047_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault015_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault030_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault001_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault029_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault002_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault052_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault038_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault009_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse011_x264.mp4",
      "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_781_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_696_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_941_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_189_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_010_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_606_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_365_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_867_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_778_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_903_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_710_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_478_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_936_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_940_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_603_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_913_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_702_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_898_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_312_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_210_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_439_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_909_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_931_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_907_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_168_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_872_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_015_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_877_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_801_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_063_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse023_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse042_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse036_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse050_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse027_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse045_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse029_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse021_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse033_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse009_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse048_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse049_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse037_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse031_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse038_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse030_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse026_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse010_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse025_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse012_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse035_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse043_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse002_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse047_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse017_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse034_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse007_x264.mp4",
"/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse003_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_597_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_938_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_576_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_906_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_909_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_015_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_621_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_888_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_722_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_182_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_781_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_401_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_025_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_452_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_868_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_641_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_891_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_884_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_896_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_879_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_778_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_905_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_911_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_931_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_881_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_885_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_150_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_014_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_018_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_704_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_895_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_801_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_930_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_365_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_070_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_831_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_247_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_289_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_939_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_910_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_310_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_798_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_168_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_251_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_871_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_041_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_686_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_923_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_780_x264.mp4",
        "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_869_x264.mp4",
         ]
dataset_root = "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/"

# Walk through the dataset and remove files not in keep_files
for root, dirs, files in os.walk(dataset_root):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path not in keep_files:
            try:
                os.remove(file_path)
                print(f"Removed: {file_path}")
            except Exception as e:
                print(f"Error removing {file_path}: {e}")


Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Temporal_Anomaly_Annotation_for_Testing_Videos.txt
Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/ReadMe-Anomaly-Detection.txt
Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_926_x264.mp4
Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_224_x264.mp4
Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_203_x264.mp4
Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_878_x264.mp4
Removed: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/

In [9]:

keep_files_list = list(keep_files)
labeled_videos = []
# Use the actual dataset_path obtained from kagglehub
actual_dataset_root = dataset_path

for i, video_path in enumerate(keep_files_list):
    # Reconstruct the path using the actual_dataset_root
    # Assuming the structure within the dataset_path is consistent
    # We need to find the part of the hardcoded path that comes after the dataset_path root
    # Example: hardcoded path /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault036_x264.mp4
    # actual_dataset_root is /kaggle/input/crimeucfdataset
    # We need to find the relative path from actual_dataset_root, which seems to be Anomaly_Dataset/Anomaly_Videos/...
    # A robust way is to find the common part and append the rest, or assume a consistent structure.
    # Given the previous printout, let's assume the structure after the root is 'Anomaly_Dataset/Anomaly_Videos/' followed by the rest.

    # Find the part of the path after the assumed root in the hardcoded path
    assumed_hardcoded_root_part = "/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/"
    relative_path_start_index = video_path.find("Anomaly_Dataset/Anomaly_Videos/")
    if relative_path_start_index != -1:
        relative_video_path = video_path[relative_path_start_index:]
        correct_video_path = os.path.join(actual_dataset_root, relative_video_path)
        label = "crime" if "Anomaly-Videos-Part-1/Assault" in correct_video_path or \
                         "Anomaly-Videos-Part-2/Explosion" in correct_video_path or \
                         "Anomaly-Videos-Part-2/Burglary" in correct_video_path else "no crime" # More robust labeling
        labeled_videos.append((correct_video_path, label))
    else:
        print(f"Warning: Could not determine relative path for {video_path}")

In [10]:

# Ensure dataset_path is defined from cell geyWFPKEJ9kD
if 'dataset_path' not in locals():
    print("Error: dataset_path is not defined. Please run cell geyWFPKEJ9kD first.")
else:
    # Construct the full path to the file you want to check
    file_to_check = os.path.join(dataset_path, "Anomaly_Dataset", "Anomaly_Videos", "Anomaly-Videos-Part-1", "Assault", "Assault037_x264.mp4")
    print(f"Checking file: {file_to_check}")
    print("Exists:", os.path.exists(file_to_check))
    print("Size:", os.path.getsize(file_to_check) if os.path.exists(file_to_check) else "N/A")

    # Optionally, print the list of files that should have been kept (defined in RbKocDzYOtKc)
    if 'keep_files' in locals():
        print("\nFirst 5 files in the keep_files list:")
        for f in keep_files[:5]:
            print(f)
    else:
        print("\n'keep_files' list is not defined. Please run cell RbKocDzYOtKc first.")

Checking file: /root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault037_x264.mp4
Exists: True
Size: 26740175

First 5 files in the keep_files list:
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault036_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault018_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault027_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault026_x264.mp4
/root/.cache/kagglehub/datasets/mission-ai/crimeucfdataset/versions/2/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/Assault037_x264.mp4


In [11]:

# Dataset splitting (Run right after creating 'labeled_videos')
import random

random.shuffle(labeled_videos)
train_size = int(0.8 * len(labeled_videos))
training_dataset = labeled_videos[:train_size]
testing_dataset = labeled_videos[train_size:]

print(f"Total videos: {len(labeled_videos)}")
print(f"Training videos: {len(training_dataset)}")
print(f"Testing videos: {len(testing_dataset)}")

Total videos: 160
Training videos: 128
Testing videos: 32


In [12]:

label_map = {"no crime": 0, "crime": 1}

mean = np.array([0.43216, 0.394666, 0.37645])
std = np.array([0.22803, 0.22145, 0.216989])

In [31]:
import cv2
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import numpy as np


class CrimeVideoDataset(Dataset):
    def __init__(self, video_label_list, mean, std, clip_len=16, num_clips=5, size=112):
        self.data = video_label_list
        self.mean = mean
        self.std = std
        self.clip_len = clip_len
        self.num_clips = num_clips
        self.size = size
        self.transform = transforms.Compose([
            transforms.ToPILImage(),   # Convert NumPy array to PIL Image for resizing
            transforms.Resize((size, size)),
            transforms.ToTensor(),     # Convert PIL Image to Tensor
            transforms.Normalize(mean=mean, std=std)  # Normalize
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, label = self.data[idx]
        frames = []
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"Error opening video file: {path}")
            return self.__getitem__((idx + 1) % len(self))  # Skip to next video

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            frames.append(self.transform(frame))            # Apply transforms on NumPy array

        cap.release()

        if len(frames) < self.clip_len:
            # Pad with black frames if video is too short
            num_padding_frames = self.clip_len - len(frames)
            padding_frames = [torch.zeros(3, self.size, self.size) for _ in range(num_padding_frames)]
            frames.extend(padding_frames)
            print(f"Warning: Video {path} is shorter than clip_len. Padded with black frames.")

        max_start = max(len(frames) - self.clip_len, 1)
        chosen_starts = np.linspace(0, max_start, num=self.num_clips, dtype=int)
        clips = []
        for s in chosen_starts:
            if s + self.clip_len <= len(frames):
                # Stack frames for clip and permute to (C, T, H, W)
                clip = torch.stack(frames[s:s+self.clip_len]).permute(1, 0, 2, 3)
                clips.append(clip)

        if len(clips) == 0:
            # If no clips extracted, return zero clip
            clips = [torch.zeros(3, self.clip_len, self.size, self.size)]
            print(f"Warning: Could not extract clips from {path}. Returning zero clip.")

        processed_clips = []
        for clip in clips:
            if clip.shape != (3, self.clip_len, self.size, self.size):
                print(f"Warning: Clip from {path} has unexpected shape {clip.shape}. Replacing with zero clip.")
                processed_clips.append(torch.zeros(3, self.clip_len, self.size, self.size))
            else:
                processed_clips.append(clip)

        if not processed_clips:
            processed_clips = [torch.zeros(3, self.clip_len, self.size, self.size) for _ in range(self.num_clips)]
            print(f"Warning: No valid clips processed for {path}. Returning batch of zero clips.")

        # Handle the "unknown" label case for prediction
        if label == "unknown":
             return torch.stack(processed_clips), torch.tensor(0) # Return a dummy label 0 for unknown
        else:
            return torch.stack(processed_clips), torch.tensor(label_map[label])

In [14]:

# Cell 4 - Function to batch extract embeddings using R3D model
def batch_r3d_embeddings(r3d_model, dataloader, device):
    r3d_model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for clips, label in dataloader:
            bs, num_clips, C, T, H, W = clips.shape
            clips = clips.view(-1, C, T, H, W).to(device)
            feat = r3d_model.stem(clips)
            for layer in [r3d_model.layer1, r3d_model.layer2, r3d_model.layer3, r3d_model.layer4]:
                feat = layer(feat)
            emb = feat.mean([-3, -2, -1]).view(bs, num_clips, -1).cpu()
            embeddings.append(emb)
            labels.append(label)
    return torch.cat(embeddings), torch.cat(labels)


In [15]:
# Cell 5 - LSTM Autoencoder scoring function
def lstmae_batch_score(lstmae, emb_seq_batch, device):
    lstmae.eval()
    scores = []
    with torch.no_grad():
        for emb_seq in emb_seq_batch:
            inp = emb_seq.unsqueeze(0).to(device)  # Add batch dim
            recon = lstmae(inp)
            mse = ((inp - recon)**2).mean().item()
            scores.append(mse)
    return np.array(scores)

In [18]:
torch.save(r3d.state_dict(), "r3d_finetuned.pth")

In [19]:

# Cell 6 - Load your trained r3d and lstmae models (modify paths accordingly)
from torchvision.models.video import r3d_18

# Load fine-tuned R3D-18
r3d = r3d_18(weights=None)
num_features = r3d.fc.in_features
r3d.fc = torch.nn.Linear(num_features, 2)
r3d.load_state_dict(torch.load('r3d_finetuned.pth', map_location=device))
r3d = r3d.to(device).eval()

# Define LSTM Autoencoder class (if not defined in prior cell)
class LSTMAutoEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.encoder = torch.nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.decoder = torch.nn.LSTM(hidden_dim, input_dim, batch_first=True)
        self.hidden_dim = hidden_dim
    def forward(self, x):
        _, (h, _) = self.encoder(x)
        z = h[-1].unsqueeze(1).repeat(1, x.size(1), 1)
        decoded, _ = self.decoder(z)
        return decoded

lstmae = LSTMAutoEncoder(input_dim=512, hidden_dim=128)
torch.save(lstmae.state_dict(), "lstmae_trained.pth")
lstmae.load_state_dict(torch.load('lstmae_trained.pth', map_location=device))
lstmae = lstmae.to(device).eval()

In [20]:



# Cell 7 - Create dataset and dataloader for testing dataset (replace testing_dataset with your list)
from torch.utils.data import DataLoader # Import DataLoader here

test_dataset = CrimeVideoDataset(testing_dataset, mean, std)
test_loader = DataLoader(test_dataset, batch_size=8, num_workers=2)

In [21]:

!sudo apt-get update
!sudo apt-get install -y libavdevice-dev libavfilter-dev libavformat-dev libavcodec-dev libavutil-dev libswscale-dev libavresample-dev
!pip install av

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,812 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,374 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease


In [22]:

import av
import os

print(av.__version__)

# Use the first video path from the keep_files list for testing
if 'keep_files' in locals() and keep_files:
    test_video_path = keep_files[0]
    try:
        container = av.open(test_video_path)  # Use a path from the keep_files list
        print(container.streams.video)
    except FileNotFoundError:
        print(f"Error: The test video file was not found at {test_video_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("Error: The 'keep_files' list is not defined or is empty. Please run the relevant cells first.")

16.0.1
(<av.VideoStream #0 h264, yuv420p 320x240 at 0x798bba94d0d0>, <av.VideoStream #2 png, rgb24 58x38 at 0x798bb9223650>)


In [23]:
!pip install --upgrade torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 163.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.

In [24]:

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=0)

In [25]:

# Cell 8 - Run embedding extraction and compute anomaly scores
emb_seqs, gt_labels = batch_r3d_embeddings(r3d, test_loader, device)
anomaly_scores = lstmae_batch_score(lstmae, emb_seqs, device)


In [42]:
# Cell 9 - Threshold anomaly scores for predictions and calculate accuracy
# The threshold determines the cutoff for classifying a video as 'crime'.
# A higher threshold will reduce False Positives (non-crime classified as crime)
# but may increase False Negatives (crime classified as non-crime).
# Tune this threshold based on the desired trade-off between False Positives and False Negatives
# and evaluate using the metrics in the next cell.
threshold = 1.7 # Currently set to 0.9

pred_labels = ['crime' if score > threshold else 'no crime' for score in anomaly_scores]
gt_labels_str = ['crime' if l==1 else 'no crime' for l in gt_labels.numpy()]
accuracy = (np.array(pred_labels) == np.array(gt_labels_str)).mean() * 100

print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 71.88%


In [43]:
from google.colab import files
import os
import traceback

uploaded = files.upload()

for filename in uploaded.keys():
    print(f'User uploaded file "{filename}"')
    # Use the uploaded filename to predict
    uploaded_video_path = filename
    if os.path.exists(uploaded_video_path):
        #print(f"Predicting on uploaded video: {uploaded_video_path}")
        # Ensure all necessary variables and functions are defined from previous cells
        # (CrimeVideoDataset, batch_r3d_embeddings, lstmae_batch_score, r3d, lstmae, mean, std, threshold, device)
        try:
            # Create a dummy dataset and dataloader for the single video
            single_video_dataset = CrimeVideoDataset([(uploaded_video_path, "unknown")], mean, std)
            single_video_loader = DataLoader(single_video_dataset, batch_size=1, shuffle=False, num_workers=0)

            # Extract embeddings
            emb_seqs, _ = batch_r3d_embeddings(r3d, single_video_loader, device)

            # Compute anomaly score
            anomaly_score = lstmae_batch_score(lstmae, emb_seqs, device)[0] # Get the single score

            # Predict based on threshold
            # The prediction is 'crime' if the anomaly_score is greater than the threshold.
            # To reduce false positives (non-crime classified as crime), you can increase the threshold
            # in cell Wzn-gzDfdlD9 and re-run this cell.
            prediction = 'crime' if anomaly_score > threshold else 'no crime'

            #print(f"Anomaly Score: {anomaly_score:.4f}") # Print the anomaly score
            print(f"Prediction: {prediction}")

        except NameError as e:
            print(f"Error: {e}. Please ensure all necessary functions and variables are defined in previous cells.")
        except Exception as e:
            print(f"An error occurred during prediction: {e}")
            traceback.print_exc() # Print the full traceback for the exception
    else:
        print(f"Error: Uploaded file not found at {uploaded_video_path}")

Saving good1.mp4 to good1 (5).mp4
User uploaded file "good1 (5).mp4"
Predicting on uploaded video: good1 (5).mp4
Anomaly Score: 0.9798
Prediction: no crime


In [35]:
import torch
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np # Import numpy

# Assuming you have your ground truth labels and predicted labels as lists or 1D tensors:
# gt_labels = [...]  # Ground truth labels, e.g. [0, 1, 0, 1, ...]
# pred_labels = [...]  # Predicted labels from your model, e.g. [0, 0, 0, 1, ...]

# Convert to CPU numpy arrays if they are tensors
gt = gt_labels.cpu().numpy() if torch.is_tensor(gt_labels) else np.array([label_map[l] for l in gt_labels_str])

# Convert predicted labels from strings to integers using the label_map
pred = np.array([label_map[p] for p in pred_labels])


# Compute confusion matrix
cm = confusion_matrix(gt, pred)
tn, fp, fn, tp = cm.ravel()

print("Confusion Matrix:")
print(cm)

# Print detailed stats, highlighting False Positives
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp} <--- False alarms!")
print(f"False Negatives (FN): {fn} <--- Missed crimes!")

# Calculate metrics
precision = precision_score(gt, pred)
recall = recall_score(gt, pred)
f1 = f1_score(gt, pred)
# Handle the case where fp + tn is zero to avoid division by zero
false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"False Alarm Rate: {false_alarm_rate:.4f} <--- Proportion of non-crime videos incorrectly flagged as crime.")

Confusion Matrix:
[[ 6 17]
 [ 2  7]]
True Positives (TP): 7
True Negatives (TN): 6
False Positives (FP): 17 <--- False alarms!
False Negatives (FN): 2 <--- Missed crimes!
Precision: 0.2917
Recall: 0.7778
F1-Score: 0.4242
False Alarm Rate: 0.7391 <--- Proportion of non-crime videos incorrectly flagged as crime.
